In [1]:
import numpy as np
import pandas as pd
import os
import glob
from datetime import datetime

In [16]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(os.path.dirname(os.path.abspath('')),"raw_data","preproc_data_first_night", "*.csv"))
df = pd.concat((pd.read_csv(f, index_col=0, header=0) for f in all_files))

In [2]:
df = pd.read_csv('/Users/jonaspieper/Documents/GitHub/TravelDashboard/raw_data/preproc24hdata.csv')

In [3]:
def get_total_per_country(df = df, from_time = '00:00:00', to_time = '23:59:59'):
    #convert epoch time to datetime
    df['dtime'] = pd.to_datetime(df['time'], unit='s')
    
    #define period you want to look at
    start = datetime.strptime(from_time, '%H:%M:%S').time()
    end = datetime.strptime(to_time, '%H:%M:%S').time()
    df = df[df['dtime'].dt.time.between(start, end)]
    
    #filter for aircraft which are below 1000m(?) and have a vertical rate > 3 OR < -3
    df = df[((df['geo_altitude'] < 1000) & (df['vertical_rate'] > 3)) | ((df['geo_altitude'] < 1000) & (df['vertical_rate'] < -3))]
    
    #create new column which devides into starting or not
    df['starting'] = [True if x > 3 else False for x in df['vertical_rate']]
    
    #only keep one row per callsign (with the lowest altitude because closest to airport)
    df = df.sort_values(by=['geo_altitude'],ascending=True).groupby('callsign',as_index=False).first()
    
    #create df which shows numbers of passenger starting and landing per county
    sum_starting_landing = df.groupby(['country_cc','starting']).agg({'avg_no_seats': 'sum'})
    sum_starting_landing = sum_starting_landing.rename(columns = {'avg_no_seats':'total_passenger'})

    
    
    
    #devide df into starting and landing to calculate difference between arriving and departing passengers
    starting = df[df['starting'] == True].groupby('country_cc').sum('avg_no_seats')
    landing = df[df['starting'] == False].groupby('country_cc').sum('avg_no_seats')

    #create 2nd df for total number per country
    sum_pass = landing.avg_no_seats.sub(starting['avg_no_seats'], fill_value = 0)
    

    return sum_pass, sum_starting_landing

In [4]:
sum_pass, sum_starting_landing = get_total_per_country()

In [ ]:
# find parameters for fitting numbers
from_time = '00:00:00' 
to_time = '23:59:59'
df = pd.concat((pd.read_csv(f, index_col=0, header=0) for f in all_files))
df['dtime'] = pd.to_datetime(df['time'], unit='s')
    
#define period you want to look at
start = datetime.strptime(from_time, '%H:%M:%S').time()
end = datetime.strptime(to_time, '%H:%M:%S').time()
df = df[df['dtime'].dt.time.between(start, end)]

#filter for aircraft which are below 1000m(?) and have a vertical rate > 3 OR < -3
df = df[((df['geo_altitude'] < 3500) & (df['vertical_rate'] > 4)) | ((df['geo_altitude'] < 3000) & (df['vertical_rate'] < -4))]

#create new column which devides into starting or not
df['starting'] = [True if x > 0 else False for x in df['vertical_rate']]

starting = df[df['starting'] == True].sort_values(by=['geo_altitude'],ascending=True).groupby('callsign',as_index=False).first()
landing = df[df['starting'] == False].sort_values(by=['geo_altitude'],ascending=True).groupby('callsign',as_index=False).first()
print(sum(starting['callsign'].isin(landing['callsign']))/len(starting))

starting = starting.groupby('country_cc').sum('avg_no_seats')
landing = landing.groupby('country_cc').sum('avg_no_seats')

#only keep one row per callsign (with the lowest altitude because closest to airport)
#df = df.sort_values(by=['geo_altitude'],ascending=True).groupby('callsign',as_index=False).first()

#create df which shows numbers of passenger starting and landing per county



#devide df into starting and landing to calculate difference between arriving and departing passengers


sum_pass = landing.avg_no_seats.sub(starting['avg_no_seats'], fill_value = 0)
print(sum_pass.sum())
print(starting.shape, landing.shape)
